In [41]:
# Load API key.
from dotenv import load_dotenv
load_dotenv()

True

In [65]:
# First we initialize the model we want to use.
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash")


# For this tutorial we will use custom tool that returns pre-defined values for weather in two cities (NYC & SF)
from typing import Literal

from langchain_core.tools import tool


@tool
def get_weather(location: str):
    """Use this to get weather information from a given location."""
    if location.lower() in ["nyc", "new york"]:
        return "It might be cloudy in nyc"
    elif location.lower() in ["sf", "san francisco"]:
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown Location")


tools = [get_weather]

# We need a checkpointer to enable human-in-the-loop patterns
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(
    model, tools=tools, interrupt_before=["tools"], checkpointer=memory
)

In [66]:
def print_stream(stream):
    """A utility to pretty print the stream."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

In [67]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": "42"}}
inputs = {"messages": [("user", "what is the weather in SF, CA?")]}

print_stream(graph.stream(inputs, config, stream_mode="values"))

================================ Human Message =================================

what is the weather in SF, CA?
================================== Ai Message ==================================
Tool Calls:
  get_weather (362ea5c6-abe2-4230-9a76-fe86bdceaedf)
 Call ID: 362ea5c6-abe2-4230-9a76-fe86bdceaedf
  Args:
    location: SF, CA


In [68]:
snapshot = graph.get_state(config)
print("Next step: ", snapshot.next)

Next step:  ('tools',)


In [69]:
print_stream(graph.stream(None, config, stream_mode="values"))

================================== Ai Message ==================================
Tool Calls:
  get_weather (362ea5c6-abe2-4230-9a76-fe86bdceaedf)
 Call ID: 362ea5c6-abe2-4230-9a76-fe86bdceaedf
  Args:
    location: SF, CA
================================= Tool Message =================================
Name: get_weather

Error: AssertionError('Unknown Location')
 Please fix your mistakes.


================================== Ai Message ==================================

I am sorry, I cannot fulfill this request. The location is invalid. Please provide a valid location.


In [73]:
state = graph.get_state(config)
state.values

{'messages': [HumanMessage(content='what is the weather in SF, CA?', additional_kwargs={}, response_metadata={}, id='3ba5ca7c-09b0-4965-bfdf-e2524b336d53'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "SF, CA"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-7aa88385-cb96-47de-bf5f-f9986a6801ef-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': '362ea5c6-abe2-4230-9a76-fe86bdceaedf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 25, 'output_tokens': 7, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="Error: AssertionError('Unknown Location')\n Please fix your mistakes.", name='get_weather', id='9f6377ed-1a0a-4200-8fba-19de89a5f2ff', tool_call_id='362ea5c6-abe2-4230-9a76-fe86bdceaedf', status='error'),
  AIMessage(content='I am sorry, I cannot fulfil

In [71]:
state = graph.get_state(config)
messages = state.values["messages"]

# Find the message with tool_calls
for message in messages:
    if hasattr(message, 'tool_calls') and message.tool_calls:
        message.tool_calls[0]["args"] = {"location": "San Francisco"}
        break

# Update the state with the modified messages
graph.update_state(config, {"messages": messages})

{'configurable': {'thread_id': '42',
  'checkpoint_ns': '',
  'checkpoint_id': '1f006055-3234-6d2e-8004-82167a94226c'}}

In [ ]:
state = graph.get_state(config)

last_message = state.values["messages"][-1]
last_message.tool_calls[0]["args"] = {"location": "San Francisco"}

graph.update_state(config, {"messages": [last_message]})

IndexError: list index out of range

In [21]:
last_message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-ff17cbb7-3961-4fe6-9974-fe422b2b859b-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'SF, CA'}, 'id': '7c5ddbc3-82fd-4a99-9eed-05480c63fec2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 25, 'output_tokens': 7, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}})

In [72]:
print_stream(graph.stream(None, config, stream_mode="values"))

================================== Ai Message ==================================

I am sorry, I cannot fulfill this request. The location is invalid. Please provide a valid location.
